In [1]:
lemmas=dict()
lemma = open('new_lemmatizer.csv')

for line in lemma:
    val=list()
    roots = line.split(',')
    end=len(roots)
    if roots.count('\n') > 0:
        roots.remove('\n')
    #if roots.count('') > 0:
    #    end = roots.index('')
    #val.clear()
    i=0;
    for r in roots:
        i=i+1
        if r != '' and i > 1:
            val.append((r,0))
    lemmas[roots[0]]=val#roots[1:end]
print(lemmas['abactum'])

[('abigo', 0), ('abactus', 0)]


In [2]:

# Please setup the variable as desired


input_files_loc='input_folder'
output_files_loc='output_folder'
n_gram=2

In [3]:
from operator import add
from __future__ import print_function
import os
import time;

localtime = time.asctime( time.localtime(time.time()) )

dummy=sc.emptyRDD()
txtFile=None

collect=list()

def do_word_co_occurence(n_gram):
    def do_bi_gram(x):
        init = x.split('> ')
        if len(init) == 2:
            a=init[1]
            init[0]=init[0]+'>'
            a=a.lower().replace('?','')\
                .replace(',','').replace('.','')\
                .replace('j','v').replace('i','u')
            words = a.split(' ')
            app = []
            words.sort()
            for word in words:
                if word in lemmas:
                    words.remove(word)
                    app=app+lemmas[word]
            words=words+app
            #print(words)
            for i in range(0,len(words)):
                for j in range(0,len(words)):
                    if j > i and not (isinstance(words[i],tuple) and isinstance(words[j],tuple)):
                        if isinstance(words[i],tuple):
                            yield ((words[i][0],words[j]),init[0])
                        elif isinstance(words[j],tuple):
                            yield ((words[i],words[j][0]),init[0])
                        else:
                            yield ((words[i],words[j]),init[0])


    def do_tri_gram(x):
        init = x.split('> ')
        if len(init) == 2:
            a=init[1]
            init[0]=init[0]+'>'
            a=a.lower().replace('?','')\
                .replace(',','').replace('.','')\
                .replace('j','v').replace('i','u')
            words = a.split(' ')
            app = []
            words.sort()
            for word in words:
                if word in lemmas:
                    words.remove(word)
                    app=app+lemmas[word]
            words=words+app
            for i in range(0,len(words)):
                for j in range(i+1,len(words)):
                    for k in range(j+1,len(words)):
                        
                        # if all of the words for trigram are not tuple there will be no else
                        if not (isinstance(words[i],tuple) and isinstance(words[j],tuple) and isinstance(words[k],tuple)):
                            
                            #first tuple
                            if isinstance(words[i],tuple) and not isinstance(words[j],tuple) and not isinstance(words[k],tuple):
                                yield ((words[i][0],words[j],words[k]),init[0])
                            
                            #second tuple
                            elif isinstance(words[j],tuple) and not isinstance(words[i],tuple) and not isinstance(words[k],tuple):
                                yield ((words[i],words[j][0],words[k]),init[0])
                            
                            #third tuple
                            elif isinstance(words[k],tuple) and not isinstance(words[i],tuple) and not isinstance(words[j],tuple):
                                yield ((words[i],words[j],words[k][0]),init[0])
                            
                            else:
                                #check no two tuple
                                if not (isinstance(words[i],tuple) or isinstance(words[j],tuple))\
                                and not (isinstance(words[j],tuple) or isinstance(words[k],tuple))\
                                and not (isinstance(words[i],tuple) or isinstance(words[k],tuple)):
                                    yield ((words[i],words[j],words[k]),init[0])
    
    
    if n_gram == 2:
        print('====>2')
        return txtFile.flatMap(do_bi_gram).reduceByKey(add)
    if n_gram == 3:
        print('====>3')
        return txtFile.flatMap(do_tri_gram).reduceByKey(add)

#collect=list()
for file in os.listdir(input_files_loc):
    print(file+input_files_loc)
    txtFile=sc.textFile(os.path.join(os.path.join(os.getcwd(),input_files_loc),file))
    collect.append(do_word_co_occurence(n_gram))
#do_word_co_occurence(n_gram)
print(len(collect))
counts=sc.union(collect).reduceByKey(add)

ab1.txtinput_folder
====>2
ambrose.apologia_david_altera.tessinput_folder
====>2
2


In [4]:
lst=counts.saveAsTextFile(output_files_loc)


print("started execution at "+localtime)
print("ended execution at "+time.asctime( time.localtime(time.time()) ))

started execution at Sat May 13 23:32:00 2017
ended execution at Sat May 13 23:32:30 2017


In [ ]:
#cnt=0;
#outputFile = open('ab_out_tri',mode='a')
#for a in lst:
#    outputFile.write(str(a)+'\n')
#outputFile.close()
#for file in os.listdir(input_files_loc):
#    for line in open(os.path.join(os.path.join(os.getcwd(),input_files_loc),file)):
#        print(line)

In [ ]:
f=os.listdir(output_files_loc)
cnt=0
cnt1=0
cnt2=0
for a in f:
    if a.startswith('p'):
        for line in open(os.path.join(os.path.join(os.getcwd(),output_files_loc),a)):
            #if '<l' in line:
            cnt=cnt+1
            if '<ambrose. ap_david_altera. 2>' in line:
                cnt1=cnt1+1
            if '<ambrose. ap_david_altera. 3>' in line:
                cnt2=cnt2+1
print(cnt)
print(cnt1)
print(cnt2)

In [ ]:
#sc.stop()

In [6]:
sc